# 3. InceptionResNetV2_SE

## Run name

In [ ]:
import time

project_name = 'inclusive-images-challenge'
step_name = '3_InceptionResNetV2_SE'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

time0 = time.time()

## Important params

In [2]:
debug_rows = 10
batch_size = 64

print(debug_rows, batch_size)

10 64


## Import pkgs

In [3]:
import os
import random
import time
import re
import math
from tqdm import tqdm
import multiprocessing
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.utils import shuffle
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, jaccard_similarity_score

cpu_amount = multiprocessing.cpu_count()
random_num = np.random.randint(0, 9999)
random_num_str = '%04d' % random_num

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num_str)

cpu_amount: 7
random_num: 7935


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

print(feature_folder)
print(input_folder)
print(output_folder)
print(model_folder)
print(log_folder)
train_folder = os.path.join(input_folder, 'train')
validation_folder = os.path.join(input_folder, 'validation')
test_folder = os.path.join(input_folder, 'test')
stage_1_test_images_folder = os.path.join(input_folder, 'stage_1_test_images')

print(train_folder)
print(validation_folder)
print(test_folder)
print(stage_1_test_images_folder)

class_descriptions_csv = os.path.join(input_folder, 'class-descriptions.csv')
class_trainable_csv = os.path.join(input_folder, 'classes-trainable.csv')
stage_1_attributions_csv = os.path.join(input_folder, 'stage_1_attributions.csv')
stage_1_sample_submission_csv = os.path.join(input_folder, 'stage_1_sample_submission.csv')
train_bounding_boxes_csv = os.path.join(input_folder, 'train_bounding_boxes.csv')
train_human_labels_csv = os.path.join(input_folder, 'train_human_labels.csv')
train_machine_labels_csv = os.path.join(input_folder, 'train_machine_labels.csv')
tuning_labels_csv = os.path.join(input_folder, 'tuning_labels.csv')

print(class_descriptions_csv)
print(class_trainable_csv)
print(stage_1_attributions_csv)
print(stage_1_sample_submission_csv)
print(train_bounding_boxes_csv)
print(train_human_labels_csv)
print(train_machine_labels_csv)
print(tuning_labels_csv)

/data/kaggle-Inclusive-Images/feature
/data/kaggle-Inclusive-Images/input
/data/kaggle-Inclusive-Images/output
/data/kaggle-Inclusive-Images/model
/data/kaggle-Inclusive-Images/log
/data/kaggle-Inclusive-Images/input/train
/data/kaggle-Inclusive-Images/input/validation
/data/kaggle-Inclusive-Images/input/test
/data/kaggle-Inclusive-Images/input/stage_1_test_images
/data/kaggle-Inclusive-Images/input/class-descriptions.csv
/data/kaggle-Inclusive-Images/input/classes-trainable.csv
/data/kaggle-Inclusive-Images/input/stage_1_attributions.csv
/data/kaggle-Inclusive-Images/input/stage_1_sample_submission.csv
/data/kaggle-Inclusive-Images/input/train_bounding_boxes.csv
/data/kaggle-Inclusive-Images/input/train_human_labels.csv
/data/kaggle-Inclusive-Images/input/train_machine_labels.csv
/data/kaggle-Inclusive-Images/input/tuning_labels.csv


## Load csv

In [5]:
%%time
files = list(os.listdir(train_folder))
print(len(files))
print(files[:10])

1743042
['0642122f9be33fd1.jpg', '404410210b7bccf3.jpg', 'ed7e7b964e93dd46.jpg', 'e09f120f0e55236d.jpg', '93ab5a03abc4c571.jpg', '6e5fdc2065707bd3.jpg', '5d31490c77d80d03.jpg', 'd49a4a626f6146fd.jpg', '7435dd62e2b684dd.jpg', '09ea77347018c25d.jpg']
CPU times: user 256 ms, sys: 740 ms, total: 996 ms
Wall time: 991 ms


In [6]:
%%time
train_bounding_boxes_df = pd.read_csv(train_bounding_boxes_csv, nrows=None)

CPU times: user 18.5 s, sys: 1.6 s, total: 20.1 s
Wall time: 20.1 s


In [7]:
# %%time
# train_machine_labels_df = pd.read_csv(train_machine_labels_csv)

In [8]:
# %%time
# train_human_labels_df = pd.read_csv(train_human_labels_csv)

In [9]:
%%time
stage_1_attributions_df = pd.read_csv(stage_1_attributions_csv)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 27.7 ms


In [10]:
# %%time
# stage_1_sample_submission_df = pd.read_csv(stage_1_sample_submission_csv)

In [11]:
display('train_bounding_boxes_df', train_bounding_boxes_df.shape, train_bounding_boxes_df.head(2))
# display('train_machine_labels_df', train_machine_labels_df.shape, train_machine_labels_df.head(2))
# display('train_human_labels_df', train_human_labels_df.shape, train_human_labels_df.head(2))
display('stage_1_attributions_df', stage_1_attributions_df.shape, stage_1_attributions_df.head(2))
# display('stage_1_sample_submission_df', stage_1_sample_submission_df.shape, stage_1_sample_submission_df.head(2))

'train_bounding_boxes_df'

(14610229, 13)

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.0125,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.0250,0.276563,0.714063,0.948438,0,1,0,0,0


'stage_1_attributions_df'

(32580, 2)

,image_id,source
0,2b2b327132556c767a736b3d,Nita Ar
1,2b2b394755692f303963553d,Sathya Esarapu


In [12]:
print(len(set(train_bounding_boxes_df['LabelName'])), '--> Use this 599 labels, approximate 600 labels')
# print(len(set(train_machine_labels_df['LabelName'])))
# print(len(set(train_human_labels_df['LabelName'])))
# print(len(set(stage_1_sample_submission_df['labels'])))

599 --> Use this 599 labels, approximate 600 labels


In [13]:
labels = list(set(train_bounding_boxes_df['LabelName']))
len_labels = len(labels)
print(len_labels)
label2num = {}
num2label = {}
for i, label in enumerate(labels):
    label2num[label] = i
    num2label[i] = label
# print(label2num)
# print(num2label)

599


## Split

In [14]:
%%time

data_ids = list(set(train_bounding_boxes_df['ImageID']))
train_ids, val_ids = train_test_split(data_ids, test_size=0.05, random_state=random_num)
print('len(data_ids):\t', len(data_ids))
print('len(train_ids):\t%s\t' % len(train_ids), len(train_ids)/len(data_ids))
print('len(val_ids):\t%s\t' % len(val_ids), len(val_ids)/len(data_ids))
train_id_set = set(train_ids)
val_id_set = set(val_ids)

train_df = train_bounding_boxes_df[train_bounding_boxes_df['ImageID'].apply(lambda x: x in train_id_set)]
val_df = train_bounding_boxes_df[train_bounding_boxes_df['ImageID'].apply(lambda x: x in val_id_set)]
print('train_df.shape:\t', train_df.shape, '\t', train_df.shape[0]/train_bounding_boxes_df.shape[0])
print('val_df.shape:\t', val_df.shape, '\t\t', val_df.shape[0]/train_bounding_boxes_df.shape[0])

len(data_ids):	 1743042
len(train_ids):	1655889	 0.9499994836613231
len(val_ids):	87153	 0.050000516338676866
train_df.shape:	 (13884145, 13) 	 0.9503030376868151
val_df.shape:	 (726084, 13) 		 0.04969696231318482
CPU times: user 8.38 s, sys: 804 ms, total: 9.18 s
Wall time: 9.17 s


## Build model

In [15]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.utils import Sequence
from keras import backend as K

from skimage.io import imread
from skimage.transform import resize

Using TensorFlow backend.


In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
def data_generator(x_series, y_series, image_folder, label2num, batch_size, taget_size=200):
    len_sample = len(x_series)
    len_label = len(label2num)
    if y_series is None:
        y_data = ['/m/01g317'] * len_sample
        y_series = pd.Series(data=y_data, index=x_series.index)
    while(1):
        batch_x = np.zeros(((batch_size, taget_size, taget_size, 3)))
        batch_y = np.zeros((batch_size, len_label))
        batch_weigths = np.ones((batch_size,))
        idces = np.random.choice(len_sample, batch_size)
#         print(idces)
        for i, idx in enumerate(idces):
            try:
                file = os.path.join(image_folder, '%s.jpg' % x_series[idx])
                img_obj = imread(file, mode='constant')
                img_np = resize(img_obj, (taget_size, taget_size, 3))
                batch_x[i] = img_np
                
                num = label2num[y_series[idx]]
#                 print(i, y_series[idx], num)
                batch_y[i, num] = 1
            except Exception as ex:
                batch_weigths[i] = 0.
#                 print(x_series[idx], ex)
        
        yield batch_x, batch_y

In [18]:
# class DataGenerator(object):
#     def __init__(self, x_series, y_series, image_folder, label2num, batch_size, taget_size=200, shuffle=True, random_state=None):
#         self.x_series = x_series
#         self.y_series = y_series
#         self.image_folder = image_folder
#         self.label2num = label2num
#         self.batch_size = batch_size
#         self.taget_size = taget_size
        
#         if y_series is not None:
#             self.project_type = 'train'
#         else:
#             self.project_type = 'test'
#     def flow(self):
        

In [19]:
print(train_df.loc[:, 'ImageID'].index)

Int64Index([       0,        1,        2,        3,        4,        5,
                   6,        7,        8,        9,
            ...
            14610219, 14610220, 14610221, 14610222, 14610223, 14610224,
            14610225, 14610226, 14610227, 14610228],
           dtype='int64', length=13884145)


In [20]:
gen_train = data_generator(train_df.loc[:, 'ImageID'], train_df.loc[:, 'LabelName'], train_folder, label2num, batch_size)
gen_val = data_generator(val_df.loc[:, 'ImageID'], val_df.loc[:, 'LabelName'], train_folder, label2num, batch_size)
# gen_test = data_generator(stage_1_attributions_df.loc[:, 'image_id'], val_df.loc[:, 'LabelName'], train_folder, label2num, batch_size)

In [21]:
%%time
batch_x, batch_y = next(gen_train)
print(batch_x.shape, batch_y.shape)
batch_x, batch_y = next(gen_train)
print(batch_x.shape, batch_y.shape)
batch_x, batch_y = next(gen_train)
print(batch_x.shape, batch_y.shape)

batch_x, batch_y = next(gen_val)
print(batch_x.shape, batch_y.shape)
batch_x, batch_y = next(gen_val)
print(batch_x.shape, batch_y.shape)
batch_x, batch_y = next(gen_val)
print(batch_x.shape, batch_y.shape)

(64, 200, 200, 3) (64, 599)
(64, 200, 200, 3) (64, 599)
(64, 200, 200, 3) (64, 599)
(64, 200, 200, 3) (64, 599)
(64, 200, 200, 3) (64, 599)
(64, 200, 200, 3) (64, 599)
CPU times: user 13.2 s, sys: 19.4 s, total: 32.6 s
Wall time: 13.4 s


In [22]:
steps_per_epoch = 10000 # math.ceil(train_df.shape[0]/batch_size)
validation_steps = 10000 # math.ceil(val_df.shape[0]/batch_size)
print('steps_per_epoch:\t', steps_per_epoch)
print('validation_steps:\t', validation_steps)

steps_per_epoch:	 10000
validation_steps:	 10000


In [23]:
class ImageSequence(Sequence):
    def __init__(self, x_series, y_series, image_folder, label2num, batch_size, taget_size=200):
        self.x, self.y = x_series, y_series
        self.image_folder = image_folder
        self.label2num = label2num
        self.batch_size = batch_size
        self.taget_size = taget_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_ids = list(self.x[idx * self.batch_size:(idx + 1) * self.batch_size])
        batch_labels = list(self.y[idx * self.batch_size:(idx + 1) * self.batch_size])
        assert len(batch_image_ids) == len(batch_labels), 'Must be equal'
        len_batch = len(batch_image_ids)
        batch_sample_weigths = np.ones((batch_size,))
        
#         file_pathes = [os.path.join(self.image_folder, '%s.jpg' % file_name) for file_name in batch_image_ids]
        batch_x = np.zeros(((len_batch, self.taget_size, self.taget_size, 3)))
        for i in range(len_batch):
            img_np = np.array((len_batch, len_batch, 3))
            try:
                file = os.path.join(self.image_folder, '%s.jpg' % batch_image_ids[i])
                img_obj = imread(file, mode='constant')
                img_np = resize(img_obj, (self.taget_size, self.taget_size, 3))
            except Exception as ex:
                batch_sample_weigths[i] = 0.
#                 print(batch_image_ids[i], ex)
            batch_x[i] = img_np
        
#         batch_x = np.array([resize(imread(file, mode='constant'), (200, 200, 3)) for file in file_pathes])
        batch_y = np.zeros((len_batch, len(self.label2num)))
#         print((self.batch_size, len(self.label2num)))
#         print(batch_labels[0], self.label2num[batch_labels[0]])
        
        for i in range(len(batch_labels)):
            num = self.label2num[batch_labels[i]]
#             print(i, batch_labels[i], num)
            batch_y[i, num] = 1

        return batch_x, batch_y, batch_sample_weigths

image_ids_train, labels_train = shuffle(train_df.loc[:, 'ImageID'], train_df.loc[:, 'LabelName'])
image_ids_val, labels_train = shuffle(train_df.loc[:, 'ImageID'], train_df.loc[:, 'LabelName'])
sequence_train = ImageSequence(train_df.loc[:, 'ImageID'], train_df.loc[:, 'LabelName'], train_folder, label2num, batch_size)
sequence_val = ImageSequence(val_df.loc[:, 'ImageID'], val_df.loc[:, 'LabelName'], train_folder, label2num, batch_size)

In [24]:
%%time
for i in range(3):
    temp_sequence_train = sequence_train[i]
    print(temp_sequence_train[0].shape, temp_sequence_train[1].shape, temp_sequence_train[2].shape)
for i in range(3):
    temp_sequence_val = sequence_val[i]
    print(temp_sequence_val[0].shape, temp_sequence_val[1].shape, temp_sequence_val[2].shape)

(64, 200, 200, 3) (64, 599) (64,)
(64, 200, 200, 3) (64, 599) (64,)
(64, 200, 200, 3) (64, 599) (64,)
(64, 200, 200, 3) (64, 599) (64,)
(64, 200, 200, 3) (64, 599) (64,)
(64, 200, 200, 3) (64, 599) (64,)
CPU times: user 20.1 s, sys: 21.8 s, total: 42 s
Wall time: 14.2 s


In [25]:
# def get_lr(x):
#     lr = round(3e-4 * 0.98 ** x, 6)
#     if lr < 1e-5:
#         lr = 1e-5
#     print(lr, end='  ')
#     return lr

def get_lr(x):
    if x <= 7:
        lr = 1e-4
    elif x <= 14:
        lr = 3e-5
    else:
        lr = 1e-5
    print('%s ' % lr, end='')
    return lr
[get_lr(i) for i in range(30)]

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

callbacks = [annealer]

0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 3e-05 3e-05 3e-05 3e-05 3e-05 3e-05 3e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 1e-05 

In [26]:
# create the base pre-trained model
# base_model = VGG16(weights=None, include_top=False) # For debug
base_model = InceptionResNetV2(weights=None, include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(len_labels, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [27]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
# train the model on the new data for a few epochs
hist = model.fit_generator(
    sequence_train, 
    steps_per_epoch=None, 
    epochs=1, 
    verbose=1, 
    callbacks=callbacks, 
    validation_data=sequence_val, 
    validation_steps=None, 
    class_weight=None, 
    max_queue_size=batch_size*32, 
    workers=cpu_amount, 
    use_multiprocessing=True, 
    shuffle=True, 
    initial_epoch=0
)

Epoch 1/1
   549/216940 [..............................] - ETA: 126:19:02 - loss: 4.0044

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
weigthsFile = os.path.join(model_folder, run_name + '.h5')
print(weigthsFile)
model.save(weigthsFile)